In [1]:
import os
import sys
#set adequate flag for Theano
import theano
theano.config.gcc.cxxflags = '-march=corei7'
#check config
import keras.backend as K
K.set_image_dim_ordering('th')
#print 'Keras backend:', K.backend()
#print 'Keras image ordering operator: ', K.image_dim_ordering()
#load needed things
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.models import Sequential, Model
from keras.optimizers import SGD
from keras.layers import Input, Activation, Dense, Convolution2D, MaxPooling2D, Dropout, Flatten
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pyp
#import ROOT
import itertools
import math
from UserFunctions import *
import cPickle as pickle

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

Using cuDNN version 6021 on context None
Mapped name None to device cuda: Quadro M2000 (0000:01:00.0)
/ssd/mmelodea/miniconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [2]:
#load events
filein = open('hzz4l_vbf_selection_noDjet_m4l118-130GeV_shuffledFS_JetExtraInfo.pkl','r')
events = pickle.load( filein )
filein.close()

In [3]:
events, scales = prepareJetComponents(events)

In [4]:
#shifts the subjets towards the highest pt subjet
#eta'_i = eta_i - eta_0
#phi'_i = phi_i - phi_0
def translate(x, y):
    xShift = []
    yShift = []
    
    xCenter = x[0]
    yCenter = y[0]
    for ix, iy in zip(x, y):
        xShift.append( ix - xCenter )
        yShift.append( iy - yCenter )
    
    return xShift, yShift

In [5]:
#rotation not applied directly on eta-phi plane to preserve Lorentz invariances
#the rotation is likely done around px
def rotate(x,y,w):
    #defines subjets based on reclustering the PF candidates from PF jet
    #ak4PFJetCHS->R=0.4, so I adopt the half of it
    radius = 0.2
    maxPt = -1
    v = 0
    for ix, iy, iw, in zip(x, y, w):
        dR = math.sqrt( math.pow(ix - x[0], 2) + math.pow(iy - y[0], 2) )
        pt = iw
        if dR > radius and pt > maxPt:
            maxPt = pt
            py = pt*math.sin(iy)
            pz = pt*math.sinh(ix)
            v = np.arctan2(py,pz) + np.radians(90)

    rot_x = []
    rot_y = []
    #creates a rotation matrix
    c, s = np.cos(v), np.sin(v)
    R = np.matrix('{} {}; {} {}'.format(c, -s, s, c))
    for ix, iy, iw in zip(x, y, w):
        #original components
        px = iw*math.cos(iy)
        py = iw*math.sin(iy)
        pz = iw*math.sinh(ix)

        #transforms components
        #rot = [[py.cos(v) - pz.sin(v)],
        #       [py.sin(v) + pz.cos(v)]]
        rot = R*np.matrix([[py],[pz]])
        rix, riy = np.arcsinh(rot[1,0]/iw), np.arcsin(rot[0,0]/iw)
        
        rot_x.append(rix)
        rot_y.append(riy)
        
    return rot_x, rot_y

In [6]:
#accounts for assymetry on energy distribution
#keeps the jet average pt always in eta' positive range
def reflect(x, w):
    leftSum = 0
    rightSum = 0
    for ix, iw in zip(x, w):
        if ix > 0: 
            rightSum += iw
        elif ix < 0:
            leftSum += iw
    if(leftSum > rightSum):
        ref_x = [e*(-1) for e in x]
        return ref_x
    else:
        return x

In [48]:
#according to https://arxiv.org/abs/1704.02124 returns similar calorimeter resolution
#nx size of image in eta
#ny size of image in phi
def prepareImages(events, imc, nx, xmin, xmax, ny, ymin, ymax, pre_process):        
    list_x = [[],[],[]]
    list_y = [[],[],[]]
    list_w = [[],[],[]]

    #determines the pixel of picture
    xbins = np.linspace(xmin,xmax,nx+1)
    ybins = np.linspace(ymin,ymax,ny+1)
    #creates an empty 'th2'
    jet_images = np.zeros((len(events[imc]), 3, nx, ny))

    #group eta/phi/pt by jet
    print ik," has ", len(events[imc])," figures..."
    for iev in range(len(events[imc])):
        if(iev > 7200):
            break
        for ijet in range(len(events[imc][iev])):
            #check if there's a jet
            if(len(events[imc][iev][ijet][0]) == 0):
                continue
            # relative eta
            x = events[imc][iev][ijet][0]
            # relative phi
            y = events[imc][iev][ijet][1]
            # pt of candidate is the weight
            pt = events[imc][iev][ijet][2]
        
                
            #shifts the jet image to the center in (eta,phi) = (0,0)
            if(pre_process[0]):
                x, y = translate(x, y)

            #applies the rotation processing
            if(pre_process[1]):
                x, y = rotate(x, y, pt)
                        
            #applies the reflection processing
            if(pre_process[2]):
                x = reflect(x, pt)

            #applies normalization
            if(pre_process[3]):
                sumPt = sum(pt)
                if(sumPt == 0):
                    sumPt = 1
                nweights = []
                for iw in (pt):
                    nweights.append( iw/sumPt )
                pt = nweights
    
            list_x[ijet].append(x)
            list_y[ijet].append(y)
            list_w[ijet].append(pt)
            hist, xedges, yedges = np.histogram2d(x, y, weights=pt, bins=(xbins, ybins))
            for ix in range(0,nx):
                for iy in range(0,ny):
                    jet_images[iev,ijet,ix,iy] = hist[ix,iy]
                                        
    return jet_images, list_x, list_y, list_w, xbins, ybins

In [49]:
#check if DR is really satisfied
def passDR(eta_i, phi_i, eta_j, phi_j):
    #R = 0.4
    dij = min(1.0/(pt_i*pt_i), 1.0/(pt_j*pt_j)) * (math.pow(eta_i-eta_j,2)+math.pow(phi_i-phi_j,2)) * (1.0/(R*R))
    #dij = math.sqrt((math.pow(eta_i-eta_j,2)+math.pow(phi_i-phi_j,2)))
    
    return dij

In [ ]:
fig = pyp.figure()
fig.set_figheight(30)
fig.set_figwidth(15)

#defines pixels
nx = 25
ny = 25
       #shift,rot,reflect,norm by pt
proc = [True,True,True,False]

jets_images = {}
lists_x = {}
lists_y = {}
lists_w = {}
xbins = {}
ybins = {}

ip = 0
for ik in events:
    if(ik != 'VBF' and ik != 'HJJ'):
        continue
    jets_images[ik] = []
    lists_x[ik] = []
    lists_y[ik] = []
    lists_w[ik] = []
    xbins[ik] = []
    ybins[ik] = []

    #create and show jet images
    jets_images[ik], lists_x[ik], lists_y[ik], lists_w[ik], xbins[ik], ybins[ik] = prepareImages(events, ik, nx, -0.75, 0.75, ny, -1, 0.75, proc)

    for ich in range(3):
        all_x = np.concatenate(lists_x[ik][ich])
        all_y = np.concatenate(lists_y[ik][ich])
        all_w = np.concatenate(lists_w[ik][ich])
        all_w = 1.*all_w/len(lists_w[ik][ich]) # to get average

        ip += 1
        fig.add_subplot(8, 3, ip)
        pyp.hist2d(all_x, all_y, weights=all_w, bins=(xbins[ik],ybins[ik]), norm=mpl.colors.LogNorm(), vmin=1.e-8, vmax=1)
        pyp.colorbar()
        pyp.xlabel('eta')
        pyp.ylabel('phi')
        pyp.title( '%s - jet %i' % (ik,ich) )

pyp.tight_layout()
fig = pyp.show()

In [ ]:
# Model
conv_model = Sequential()

#conv_model.add( Convolution2D(8, 10, 10, input_shape=(3, nx, ny), border_mode='same', activation='relu') )
#conv_model.add( MaxPooling2D(pool_size=(2,2)) )
#conv_model.add( Convolution2D(8, 6, 6, border_mode='same', activation='relu') )
#conv_model.add( MaxPooling2D(pool_size=(3,3)) )
#conv_model.add( Convolution2D(8, 3, 3, border_mode='same', activation='relu') )
#conv_model.add( MaxPooling2D(pool_size=(3,3)) )
conv_model.add( Flatten(input_shape=(3, nx, ny)) )
conv_model.add( Dropout(0.30) )
conv_model.add( Dense(300, activation='relu') )
conv_model.add( Dropout(0.20) )
conv_model.add( Dense(1, activation='sigmoid') )
conv_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

conv_model.summary()

In [ ]:
#prepare inputs for model
split_factor = 0.8
X_train = []
Y_train = []
sc_train = []
X_test = []
Y_test = []
sc_test = []

for ik in jets_images:
    if(ik == 'HJJ'):
        continue
    for iev in range(len(jets_images[ik])):
        if iev < int(split_factor*len(jets_images[ik])):
            sc_train.append( scales[ik] )
            X_train.append( jets_images[ik][iev].tolist() )
            if ik == 'VBF':
                Y_train.append( 1 )
            else:
                Y_train.append( 0 )
        else:
            sc_test.append( scales[ik] )
            X_test.append( jets_images[ik][iev].tolist() )
            if ik == 'VBF':
                Y_test.append( 1 )
            else:
                Y_test.append( 0 )

X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
sc_train = np.asarray(sc_train)

X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)
sc_test = np.asarray(sc_test)
                     
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
# Train and plot ROC curves
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from scipy import interp
from keras.callbacks import ModelCheckpoint

colors = cycle(['cyan', 'indigo', 'seagreen', 'yellow', 'blue', 'darkorange', 'red', 'black', 'green', 'brown'])
lw = 1

filepath1="best_weights.hdf5"
checkpoint = ModelCheckpoint(filepath1, monitor='val_loss', verbose=0, save_best_only=True, mode='auto')
history = conv_model.fit(X_train, 
                         Y_train,
                         sample_weight=sc_train,
                         validation_data=(X_test, Y_test,sc_test),
                         epochs=10,
                         batch_size=128,
                         verbose=1,
                         callbacks=[early_stopping, checkpoint])

#conv_model.load_weights(filepath1)
sY_train = conv_model.predict(X_train)
sY_test = conv_model.predict(X_test)
# Compute ROC curve and area the curve
fpr, tpr, thresholds = roc_curve(Y_train, sY_train)
roc_auc = auc(fpr, tpr)
pyp.plot(fpr, tpr, lw=lw, color='blue', label='AUC = %0.3f' % roc_auc)
fpr, tpr, thresholds = roc_curve(Y_test, sY_test)
roc_auc = auc(fpr, tpr)
pyp.plot(fpr, tpr, lw=lw, color='red', label='AUC = %0.3f' % roc_auc)

pyp.plot([0, 1], [0, 1], linestyle='--', lw=lw, color='k', label='Luck')
pyp.xlim([0, 1.0])
pyp.ylim([0, 1.0])
pyp.xlabel('False Positive Rate')
pyp.ylabel('True Positive Rate')
pyp.title('Receiver operating characteristic example')
pyp.legend(loc="lower right")
pyp.show()